In [5]:
import os
from pymilvus import MilvusClient
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama
from langchain import hub
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import requests
import sys
import io
import json
import shutil
import sys
import datetime
import subprocess
import sys
import os
import math
import base64
from time import gmtime, strftime
import random, string
import time
import psutil
import base64
import uuid
import socket
import os
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
import torch
from torchvision import transforms
from PIL import Image
import timm
from sklearn.preprocessing import normalize
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from IPython.display import display
from pymilvus import MilvusClient
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from dotenv import load_dotenv
load_dotenv(verbose=True)
import json
import requests
from datetime import datetime, timedelta
from langchain_milvus import Milvus

DIMENSION = 384 
MILVUS_URL = "http://192.168.1.153:19530" 
TOKEN = os.environ.get("ZILLIZ_TOKEN")
slack_token = os.environ.get("SLACK_BOT_TOKEN")
client = WebClient(token=slack_token)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
OAQ3_COLLECTION_NAME = "openaqmeasurements"
TODAYS_DATE = str( datetime.today().strftime('%Y-%m-%d') ) 
YESTERDAYS_DATE = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')

os.environ["LANGCHAIN_HUB_API_URL"] = "https://api.hub.langchain.com"
os.environ["LANGCHAIN_HUB_API_KEY"] = os.environ.get("LANGCHAIN_HUB_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_HUB_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.hub.langchain.com"
os.environ["TOKENIZERS_PARALLELISM"]  = "true"

### Turn off slack warnings
os.environ["SKIP_SLACK_SDK_WARNING"] = "false"

# test query
# What is the Current Air Quality Reading in Hope?

/Users/timothyspann/Downloads/code/milvusvenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# -----------------------------------------------------------------------------
# Connect to Milvus
# https://zilliz.com/blog/a-beginners-guide-to-using-llama-3-with-ollama-milvus-langchain
# https://github.com/stephen37/ollama_local_rag/blob/main/rag_berlin_parliament.py
# https://python.langchain.com/v0.2/docs/integrations/vectorstores/milvus/
# https://api.python.langchain.com/en/latest/vectorstores/langchain_milvus.vectorstores.milvus.Milvus.html

vector_store = Milvus(
    embedding_function=embeddings,
    collection_name=OAQ3_COLLECTION_NAME,
    primary_field = "id",
    text_field="details",
    connection_args={"uri": MILVUS_URL},
)

In [3]:
def run_query() -> None:
    llm = Ollama(
        model="llama3",
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        stop=["<|eot_id|>"],
    )

    query = input("\nQuery: ")
    qa_chain = RetrievalQA.from_chain_type(
        llm, retriever=vector_store.as_retriever(collection =  OAQ3_COLLECTION_NAME, expr="parameter == 'o3'"))

    result = qa_chain({"query": query})
    
    resultforslack = str(result["result"])
    print(resultforslack)

    try:
        response = client.chat_postMessage(mrkdwn=True, channel="C06NE1FU6SE", text="", 
                                            blocks=[{"type": "section","text": {"type": "mrkdwn","text": "*" + str(query) + "*  \n\n" + str(resultforslack) +"\n" }}])

    
    except SlackApiError as e:
        # You will get a SlackApiError if "ok" is False
        print("Slack failed")

In [6]:
    while True:
        run_query()


Query:  What is the current air quality?


Based on the provided context, we can see that there are readings for both ozone (O3) and particulate matter 10 micrometers or less (PM10).

For O3, the reading at Location 902 on August 15th is 0.05 ppm.

For PM10, the reading at Location 255 on August 11th is 15.0 µg/m³.

There are no additional air quality readings provided that would indicate a current air quality status. Therefore, I can only report what we currently know:

* O3: 0.05 ppm
* PM10: 15.0 µg/m³

I don't have enough information to provide a comprehensive "current air quality" answer as there is no additional data provided beyond these two specific readings.Based on the provided context, we can see that there are readings for both ozone (O3) and particulate matter 10 micrometers or less (PM10).

For O3, the reading at Location 902 on August 15th is 0.05 ppm.

For PM10, the reading at Location 255 on August 11th is 15.0 µg/m³.

There are no additional air quality readings provided that would indicate a current air qualit

KeyboardInterrupt: Interrupted by user